<a href="https://colab.research.google.com/github/Morioh/transfer_learning_assignment/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Unzip the file
import zipfile

with zipfile.ZipFile("/content/drive/MyDrive/Alusive Documents Classification.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/data/Alusive Documents Classification")  # Extract to a folder

In [4]:
data_dir = '/content/data/Alusive Documents Classification'  # Main directory

for root, dirs, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            # Try to open the image file
            img = Image.open(file_path)
            img.verify()  # Verify that it's a valid image
        except (IOError, SyntaxError):
            print(f"Removing corrupted or unreadable image file: {file_path}")
            os.remove(file_path)

Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/Alusive Documents Classification/.DS_Store
Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/Alusive Documents Classification/Signed/.DS_Store
Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/Alusive Documents Classification/Unsigned/.DS_Store
Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/__MACOSX/Alusive Documents Classification/._.DS_Store
Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/__MACOSX/Alusive Documents Classification/Signed/._IMG_3017.jpg
Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/__MACOSX/Alusive Documents Classification/Signed/._IMG_3022.jpg
Removing corrupted or unreadable image file: /content/data/Alusive Documents Classification/__MACOSX/Alusive Documents Cl

In [5]:
import os
print(os.listdir('/content/data/Alusive Documents Classification/Alusive Documents Classification'))

['Signed', 'Unsigned']


In [6]:
# Set parameters for image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Training and validation generator with data augmentation
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Reserve 20% of data for validation
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Set up training data generator
train_data_gen = data_gen.flow_from_directory(
    '/content/data/Alusive Documents Classification',  # Path to the root folder containing signed and unsigned subfolders
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

# Set up validation data generator
val_data_gen = data_gen.flow_from_directory(
    '/content/data/Alusive Documents Classification',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Combine signed and unsigned images for training
def combined_generator(signed_gen, unsigned_gen, signed_ratio=1.0, unsigned_ratio=0.5):
    while True:
        signed_batch = signed_gen.next()
        unsigned_batch = unsigned_gen.next()

        # Adjust ratios based on imbalance (use more unsigned samples than available)
        images = np.concatenate(
            [signed_batch[0]] * int(signed_ratio * len(signed_batch[0])) +
            [unsigned_batch[0]] * int(unsigned_ratio * len(signed_batch[0]))
        )
        labels = np.concatenate(
            [signed_batch[1]] * int(signed_ratio * len(signed_batch[1])) +
            [unsigned_batch[1]] * int(unsigned_ratio * len(signed_batch[1]))
        )

        yield images, labels

train_data_gen = combined_generator(train_signed_gen, train_unsigned_gen)
val_data_gen = combined_generator(val_signed_gen, train_unsigned_gen)

# Test data generator (without augmentation, only rescaling)
test_data_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    '/content/data/Alusive Documents Classification',  # Path to your test data folder or root if combined
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Test data should generally not be shuffled
)



Found 109 images belonging to 2 classes.
Found 27 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [10]:
vgg_base = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
vgg_base.trainable = False

# Add custom classification layers
model = tf.keras.Sequential([
    vgg_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (signed or unsigned)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_data_gen,
    epochs=10,
    validation_data=val_data_gen
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 98s 24s/step - accuracy: 0.7967 - loss: 0.3749 - val_accuracy: 1.0000 - val_loss: 1.0329e-26
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 90s 22s/step - accuracy: 1.0000 - loss: 1.0809e-22 - val_accuracy: 1.0000 - val_loss: 3.8165e-38
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 97s 24s/step - accuracy: 1.0000 - loss: 2.7679e-28 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 91s 23s/step - accuracy: 1.0000 - loss: 1.3466e-37 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 148s 28s/step - accuracy: 1.0000 - loss: 1.8896e-37 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 95s 28s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 140s 23s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 144s 22s/step - accuracy: 1.0000 - loss: 2.613

In [12]:
test_loss, test_accuracy = model.evaluate(test_data_gen)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 91s 17s/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 100.00%
Test Loss: 0.0000


In [14]:
# Predict on test data
predictions = model.predict(test_data_gen)
predicted_labels = np.where(predictions > 0.25, 1, 0)  # Assuming sigmoid activation, threshold at 0.5
true_labels = test_data_gen.classes

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", conf_matrix)

# Assuming `true_labels` contains only one class
# Specify labels for both classes (Unsigned=0, Signed=1)
class_report = classification_report(true_labels, predicted_labels, target_names=['Unsigned', 'Signed'], labels=[0, 1], zero_division=0)
print("Classification Report:\n", class_report)

5/5 ━━━━━━━━━━━━━━━━━━━━ 90s 17s/step
Confusion Matrix:
 [[136]]
Classification Report:
               precision    recall  f1-score   support

    Unsigned       1.00      1.00      1.00       136
      Signed       0.00      0.00      0.00         0

    accuracy                           1.00       136
   macro avg       0.50      0.50      0.50       136
weighted avg       1.00      1.00      1.00       136



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [8]:
# Load the RASNET/ResNet model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Start building your custom model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),         # Reduces dimensions while retaining key features
    layers.Dense(128, activation='relu'),    # Add a fully connected layer
    layers.Dropout(0.5),                     # Dropout layer to reduce overfitting
    layers.Dense(1, activation='sigmoid')    # Output layer for binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
history = model.fit(
    train_data_gen,
    epochs=10,
    validation_data=val_data_gen
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 7s/step - accuracy: 0.7286 - loss: 0.3971 - val_accuracy: 1.0000 - val_loss: 0.0046
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 27s 6s/step - accuracy: 1.0000 - loss: 0.0064 - val_accuracy: 1.0000 - val_loss: 4.9906e-04
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 27s 6s/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 1.3100e-04
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 44s 8s/step - accuracy: 1.0000 - loss: 4.1468e-04 - val_accuracy: 1.0000 - val_loss: 5.6161e-05
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 10s/step - accuracy: 1.0000 - loss: 2.0041e-04 - val_accuracy: 1.0000 - val_loss: 3.3393e-05
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 31s 8s/step - accuracy: 1.0000 - loss: 2.1452e-04 - val_accuracy: 1.0000 - val_loss: 2.2703e-05
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 31s 8s/step - accuracy: 1.0000 - loss: 1.1360e-04 - val_accuracy: 1.0000 - val_loss: 1.7517e-05
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 6s/step - accuracy: 1.0000 - loss: 1.4129e-04 - val_accuracy: 

In [10]:
test_loss, test_accuracy = model.evaluate(test_data_gen)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 5s/step - accuracy: 1.0000 - loss: 1.5528e-05
Test Accuracy: 100.00%
Test Loss: 0.0000


In [13]:
# Load EfficientNetB0 without the top classification layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model weights during initial training

# Build the custom model on top of EfficientNetB0
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),         # Reduce spatial dimensions
    layers.Dense(128, activation='relu'),    # Add a fully connected layer
    layers.Dropout(0.5),                     # Dropout for regularization
    layers.Dense(1, activation='sigmoid')    # Output layer for binary classification
])

model.compile(
    optimizer='adam',                 # You may adjust this optimizer as needed
    loss='binary_crossentropy',       # Suitable for binary classification
    metrics=['accuracy']
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
history = model.fit(
    train_data_gen,
    epochs=10,
    validation_data=val_data_gen
)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.7731 - loss: 0.6033 - val_accuracy: 1.0000 - val_loss: 0.3579
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 1.0000 - loss: 0.3150 - val_accuracy: 1.0000 - val_loss: 0.1830
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 1.0000 - loss: 0.1557 - val_accuracy: 1.0000 - val_loss: 0.0905
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.0786 - val_accuracy: 1.0000 - val_loss: 0.0464
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 1.0000 - loss: 0.0470 - val_accuracy: 1.0000 - val_loss: 0.0252
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.0288 - val_accuracy: 1.0000 - val_loss: 0.0147
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 1.0000 - loss: 0.0145 - val_accuracy: 1.0000 - val_loss: 0.0092
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 1.0000 - loss: 0.0094 - val_accuracy: 1.0000 - val_loss: 0.0062
Epoch 9/

In [15]:
test_loss, test_accuracy = model.evaluate(test_data_gen)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 1.0000 - loss: 0.0033
Test Accuracy: 100.00%
Test Loss: 0.0033


In [18]:
base_model.trainable = True

# Freeze the first few layers if desired (e.g., first 100 layers)
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [20]:
history = model.fit(
    train_data_gen,
    epochs=5,
    validation_data=val_data_gen
)

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 60s 6s/step - accuracy: 0.0386 - loss: 3.2679 - val_accuracy: 1.0000 - val_loss: 0.0086
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.0206 - loss: 2.9463 - val_accuracy: 1.0000 - val_loss: 0.0175
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.0402 - loss: 2.7594 - val_accuracy: 1.0000 - val_loss: 0.0256
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.0000e+00 - loss: 2.5607 - val_accuracy: 1.0000 - val_loss: 0.0232
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.0000e+00 - loss: 2.4135 - val_accuracy: 1.0000 - val_loss: 0.0114


In [21]:
test_loss, test_accuracy = model.evaluate(test_data_gen)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 1.0000 - loss: 0.0112
Test Accuracy: 100.00%
Test Loss: 0.0110


In [22]:
# Predict on test data
predictions = model.predict(test_data_gen)
predicted_labels = np.where(predictions > 0.25, 1, 0)  # Assuming sigmoid activation, threshold at 0.5
true_labels = test_data_gen.classes

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", conf_matrix)

# Assuming `true_labels` contains only one class
# Specify labels for both classes (Unsigned=0, Signed=1)
class_report = classification_report(true_labels, predicted_labels, target_names=['Unsigned', 'Signed'], labels=[0, 1], zero_division=0)
print("Classification Report:\n", class_report)

5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step
Confusion Matrix:
 [[136]]
Classification Report:
               precision    recall  f1-score   support

    Unsigned       1.00      1.00      1.00       136
      Signed       0.00      0.00      0.00         0

    accuracy                           1.00       136
   macro avg       0.50      0.50      0.50       136
weighted avg       1.00      1.00      1.00       136



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
